In [1]:
import pandas as pd

# Direct download link
url = 'C:/Users/Mehdi Abbas/Downloads/churnpredict/Employee (1).csv'

# Load the data.csv
data = pd.read_csv(url)

data.head()


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [2]:
Churn_data=data.copy()
Churn_data['Churn']=Churn_data['LeaveOrNot'].map({1:'LEAVE',0:'STAY'})
Churn_data.head()


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,Churn
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0,STAY
1,Bachelors,2013,Pune,1,28,Female,No,3,1,LEAVE
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0,STAY
3,Masters,2016,Bangalore,3,27,Male,No,5,1,LEAVE
4,Masters,2017,Pune,3,24,Male,Yes,2,1,LEAVE


In [3]:
Churn_data.Churn.value_counts(normalize=True)

Churn
STAY     0.656136
LEAVE    0.343864
Name: proportion, dtype: float64

In [4]:
# Capitalize all Column Names

Churn_data.columns = Churn_data.columns.str.upper()

In [5]:
Churn_data.info()

# OUR DATATYPE FOR TOTAL_CHARGES IS NOT CORRECT

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   EDUCATION                  4653 non-null   object
 1   JOININGYEAR                4653 non-null   int64 
 2   CITY                       4653 non-null   object
 3   PAYMENTTIER                4653 non-null   int64 
 4   AGE                        4653 non-null   int64 
 5   GENDER                     4653 non-null   object
 6   EVERBENCHED                4653 non-null   object
 7   EXPERIENCEINCURRENTDOMAIN  4653 non-null   int64 
 8   LEAVEORNOT                 4653 non-null   int64 
 9   CHURN                      4653 non-null   object
dtypes: int64(5), object(5)
memory usage: 363.6+ KB


In [6]:
Churn_data.isna().sum()

EDUCATION                    0
JOININGYEAR                  0
CITY                         0
PAYMENTTIER                  0
AGE                          0
GENDER                       0
EVERBENCHED                  0
EXPERIENCEINCURRENTDOMAIN    0
LEAVEORNOT                   0
CHURN                        0
dtype: int64

In [7]:
# Splitting features and target variable
X = Churn_data.drop(columns=['CHURN','LEAVEORNOT'])
y = Churn_data['CHURN']

In [8]:
import numpy as np
X.select_dtypes(include=[np.float64,np.int64]).head()

,JOININGYEAR,PAYMENTTIER,AGE,EXPERIENCEINCURRENTDOMAIN
0,2017,3,34,0
1,2013,1,28,3
2,2014,3,38,2
3,2016,3,27,5
4,2017,3,24,2


In [9]:
X['PAYMENTTIER'].map({1: 'Tier 1', 2: 'Tier 2', 3: 'Tier 3'})

0       Tier 3
1       Tier 1
2       Tier 3
3       Tier 3
4       Tier 3
         ...  
4648    Tier 3
4649    Tier 2
4650    Tier 3
4651    Tier 3
4652    Tier 3
Name: PAYMENTTIER, Length: 4653, dtype: object

In [10]:
X.head()

,EDUCATION,JOININGYEAR,CITY,PAYMENTTIER,AGE,GENDER,EVERBENCHED,EXPERIENCEINCURRENTDOMAIN
0,Bachelors,2017,Bangalore,3,34,Male,No,0
1,Bachelors,2013,Pune,1,28,Female,No,3
2,Bachelors,2014,New Delhi,3,38,Female,No,2
3,Masters,2016,Bangalore,3,27,Male,No,5
4,Masters,2017,Pune,3,24,Male,Yes,2


In [11]:
X.select_dtypes(include=[object]).head()

,EDUCATION,CITY,GENDER,EVERBENCHED
0,Bachelors,Bangalore,Male,No
1,Bachelors,Pune,Female,No
2,Bachelors,New Delhi,Female,No
3,Masters,Bangalore,Male,No
4,Masters,Pune,Male,Yes


In [12]:
categorical_features = X.select_dtypes(include=[object]).columns

categorical_features= list(categorical_features)+['PAYMENTTIER']

print('\n','Categorical Features','\n', categorical_features,'\n')


 Categorical Features 
 ['EDUCATION', 'CITY', 'GENDER', 'EVERBENCHED', 'PAYMENTTIER'] 



In [13]:
numerical_features = list(X.select_dtypes(include=[np.float64,np.int64]).columns.difference(['PAYMENTTIER']))

print('\n','Numerical Features','\n', numerical_features,'\n')


 Numerical Features 
 ['AGE', 'EXPERIENCEINCURRENTDOMAIN', 'JOININGYEAR'] 



In [15]:
from sklearn.model_selection import train_test_split

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

In [16]:
print('Train Data','\n',y_train.value_counts(normalize=True),'\n','\n','Test Data','\n', y_test.value_counts(normalize=True))

Train Data 
 CHURN
STAY     0.656099
LEAVE    0.343901
Name: proportion, dtype: float64 
 
 Test Data 
 CHURN
STAY     0.656284
LEAVE    0.343716
Name: proportion, dtype: float64


In [17]:
def summarize_cat(data,categorical_features):
  results=[]

  for column in data[categorical_features]:
      # Get the unique members of the column
      members = data[column].unique().tolist()
      # Append the column name and its unique members to the results list
      results.append([column, members])

  return pd.DataFrame(results, columns=['Column Name', 'Members'])

# Create a DataFrame from the results list
summarize_cat(X_train,categorical_features)

,Column Name,Members
0,EDUCATION,"[Bachelors, Masters, PHD]"
1,CITY,"[Bangalore, Pune, New Delhi]"
2,GENDER,"[Male, Female]"
3,EVERBENCHED,"[No, Yes]"
4,PAYMENTTIER,"[3, 2, 1]"


In [18]:
summarize_cat(Churn_data,categorical_features).to_dict()

{'Column Name': {0: 'EDUCATION',
  1: 'CITY',
  2: 'GENDER',
  3: 'EVERBENCHED',
  4: 'PAYMENTTIER'},
 'Members': {0: ['Bachelors', 'Masters', 'PHD'],
  1: ['Bangalore', 'Pune', 'New Delhi'],
  2: ['Male', 'Female'],
  3: ['No', 'Yes'],
  4: [3, 1, 2]}}

In [19]:
# EXPORTING FOR DE

my_feature_dict = {'CATEGORICAL' : summarize_cat(Churn_data,categorical_features).to_dict(), 'NUMERICAL' : {'Column Name': numerical_features}}

my_feature_dict.get('NUMERICAL')

{'Column Name': ['AGE', 'EXPERIENCEINCURRENTDOMAIN', 'JOININGYEAR']}

In [20]:
my_feature_dict

{'CATEGORICAL': {'Column Name': {0: 'EDUCATION',
   1: 'CITY',
   2: 'GENDER',
   3: 'EVERBENCHED',
   4: 'PAYMENTTIER'},
  'Members': {0: ['Bachelors', 'Masters', 'PHD'],
   1: ['Bangalore', 'Pune', 'New Delhi'],
   2: ['Male', 'Female'],
   3: ['No', 'Yes'],
   4: [3, 1, 2]}},
 'NUMERICAL': {'Column Name': ['AGE',
   'EXPERIENCEINCURRENTDOMAIN',
   'JOININGYEAR']}}

In [21]:
import pickle

# save dictionary to person_data.pkl file
with open('my_feature_dict.pkl', 'wb') as fp:
    pickle.dump(my_feature_dict, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [22]:
from sklearn.pipeline import Pipeline

transform_PAYMENTTIER = lambda x: x.assign(PAYMENTTIER=x['PAYMENTTIER'].map({1: 'teir 1', 2: 'teir 2', 3: 'teir 3'}))

from sklearn.preprocessing import FunctionTransformer

preprocessor_stage_1 = Pipeline(steps=[
    ('transform_PT', FunctionTransformer(transform_PAYMENTTIER)),
])

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

pipeline_num = Pipeline(steps=[
    ('scale_data', StandardScaler()),
    ('simple_imputer1', SimpleImputer(strategy='constant',fill_value=0)),
])

from sklearn.preprocessing import OneHotEncoder

pipeline_cat = Pipeline(steps=[
    ('OneHotEncode', OneHotEncoder(handle_unknown="ignore"))
])

from sklearn.compose import ColumnTransformer

preprocessor_stage_2 = ColumnTransformer(
    transformers=[
        ('cat', pipeline_cat, categorical_features),  # Categorical columns
        ('num', pipeline_num, numerical_features),     # Numerical columns
    ],remainder='drop')

preprocessor_stack = Pipeline(steps=[
    ('preprocessor_stage_1', preprocessor_stage_1),
    ('preprocessor_stage_2', preprocessor_stage_2)
])

# BECAUSE WE DIDN'T SPECIFY CUSTOMERID IN ANY OF CATEGORICAL OR NUMERICAL FEATURES (REMAINDER='drop') REMOVE IT OUT OF PIPELINE

In [23]:
preprocessor_stack

Pipeline(steps=[('preprocessor_stage_1',
                 Pipeline(steps=[('transform_PT',
                                  FunctionTransformer(func=<function <lambda> at 0x00000296AE724A40>))])),
                ('preprocessor_stage_2',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('OneHotEncode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['EDUCATION', 'CITY',
                                                   'GENDER', 'EVERBENCHED',
                                                   'PAYMENTTIER']),
                                                 ('num',
                                                  Pipeline(steps=[('scale_data',
                                                                   StandardScaler()),
                                                                  ('simple_imputer1',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['AGE',
                                                   'EXPERIENCEINCURRENTDOMAIN',
                                                   'JOININGYEAR'])]))])

In [24]:
X_train.PAYMENTTIER.value_counts()

PAYMENTTIER
3    2789
2     737
1     196
Name: count, dtype: int64

In [25]:
preprocessor_stage_1.fit_transform(X_train).PAYMENTTIER.value_counts()

PAYMENTTIER
teir 3    2789
teir 2     737
teir 1     196
Name: count, dtype: int64

In [26]:
preprocessor_stack.fit(X_train)

Pipeline(steps=[('preprocessor_stage_1',
                 Pipeline(steps=[('transform_PT',
                                  FunctionTransformer(func=<function <lambda> at 0x00000296AE724A40>))])),
                ('preprocessor_stage_2',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('OneHotEncode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['EDUCATION', 'CITY',
                                                   'GENDER', 'EVERBENCHED',
                                                   'PAYMENTTIER']),
                                                 ('num',
                                                  Pipeline(steps=[('scale_data',
                                                                   StandardScaler()),
                                                                  ('simple_imputer1',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['AGE',
                                                   'EXPERIENCEINCURRENTDOMAIN',
                                                   'JOININGYEAR'])]))])

In [27]:
pd.DataFrame(preprocessor_stack.transform(X_train),columns=preprocessor_stack[-1].get_feature_names_out())

,cat__EDUCATION_Bachelors,cat__EDUCATION_Masters,cat__EDUCATION_PHD,cat__CITY_Bangalore,cat__CITY_New Delhi,cat__CITY_Pune,cat__GENDER_Female,cat__GENDER_Male,cat__EVERBENCHED_No,cat__EVERBENCHED_Yes,cat__PAYMENTTIER_teir 1,cat__PAYMENTTIER_teir 2,cat__PAYMENTTIER_teir 3,num__AGE,num__EXPERIENCEINCURRENTDOMAIN,num__JOININGYEAR
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.293705,-0.584442,0.505958
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.917779,0.056104,-1.094474
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.501729,1.337194,-1.094474
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.501729,1.337194,-0.027520
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.917779,0.056104,-0.560997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3717,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,-0.709754,0.696649,-1.627951
3718,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.293705,-1.865533,-1.627951
3719,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-1.125803,-0.584442,-0.560997
3720,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-1.125803,-0.584442,1.039435


In [28]:
y_train

380      STAY
1560    LEAVE
691      STAY
247      STAY
452      STAY
        ...  
4030    LEAVE
2940     STAY
1334     STAY
1579     STAY
2775    LEAVE
Name: CHURN, Length: 3722, dtype: object

In [29]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_stack),
    ('classifier', RandomForestClassifier())
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor_stage_1',
                                  Pipeline(steps=[('transform_PT',
                                                   FunctionTransformer(func=<function <lambda> at 0x00000296AE724A40>))])),
                                 ('preprocessor_stage_2',
                                  ColumnTransformer(transformers=[('cat',
                                                                   Pipeline(steps=[('OneHotEncode',
                                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                                   ['EDUCATION',
                                                                    'CITY',
                                                                    'GENDER',
                                                                    'EVERBENCHED',
                                                                    'PAYMENTTIER']),
                                                                  ('num',
                                                                   Pipeline(steps=[('scale_data',
                                                                                    StandardScaler()),
                                                                                   ('simple_imputer1',
                                                                                    SimpleImputer(fill_value=0,
                                                                                                  strategy='constant'))]),
                                                                   ['AGE',
                                                                    'EXPERIENCEINCURRENTDOMAIN',
                                                                    'JOININGYEAR'])]))])),
                ('classifier', RandomForestClassifier())])

In [30]:
X_train.head()

,EDUCATION,JOININGYEAR,CITY,PAYMENTTIER,AGE,GENDER,EVERBENCHED,EXPERIENCEINCURRENTDOMAIN
380,Bachelors,2016,Bangalore,3,28,Male,No,2
1560,Bachelors,2013,Bangalore,3,25,Female,No,3
691,Bachelors,2013,Pune,3,27,Male,No,5
247,Bachelors,2015,Bangalore,3,27,Male,No,5
452,Bachelors,2014,Bangalore,3,25,Female,No,3


In [31]:
# Checking Training Accuracy
y_train_pred = pipeline.predict(X_train)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_train,y_train_pred))
print("\nClassification Report:\n", classification_report(y_train,y_train_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_train,y_train_pred))

Accuracy: 0.9314884470714669

Classification Report:
               precision    recall  f1-score   support

       LEAVE       0.95      0.85      0.89      1280
        STAY       0.92      0.98      0.95      2442

    accuracy                           0.93      3722
   macro avg       0.94      0.91      0.92      3722
weighted avg       0.93      0.93      0.93      3722


Confusion Matrix:
 [[1085  195]
 [  60 2382]]


In [32]:
# CREATING A TEST

my_pred_array=X_test.iloc[25:26:]

my_pred_array

,EDUCATION,JOININGYEAR,CITY,PAYMENTTIER,AGE,GENDER,EVERBENCHED,EXPERIENCEINCURRENTDOMAIN
1741,Bachelors,2016,New Delhi,3,28,Male,No,4


In [33]:
pd.DataFrame(preprocessor_stack.transform(my_pred_array),columns=preprocessor_stack[1].get_feature_names_out())

,cat__EDUCATION_Bachelors,cat__EDUCATION_Masters,cat__EDUCATION_PHD,cat__CITY_Bangalore,cat__CITY_New Delhi,cat__CITY_Pune,cat__GENDER_Female,cat__GENDER_Male,cat__EVERBENCHED_No,cat__EVERBENCHED_Yes,cat__PAYMENTTIER_teir 1,cat__PAYMENTTIER_teir 2,cat__PAYMENTTIER_teir 3,num__AGE,num__EXPERIENCEINCURRENTDOMAIN,num__JOININGYEAR
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.293705,0.696649,0.505958


In [34]:
my_pred_array.to_json()

'{"EDUCATION":{"1741":"Bachelors"},"JOININGYEAR":{"1741":2016},"CITY":{"1741":"New Delhi"},"PAYMENTTIER":{"1741":3},"AGE":{"1741":28},"GENDER":{"1741":"Male"},"EVERBENCHED":{"1741":"No"},"EXPERIENCEINCURRENTDOMAIN":{"1741":4}}'

In [35]:
# USING PIPELINE TO DO ALL TOGHETHER (PREPROCESSING FOLLOWED BY MODEL PREDICT)

# SINGLE PREDICTION

y_pred = pipeline.predict(my_pred_array)

y_pred


array(['STAY'], dtype=object)

In [36]:
# USING PIPELINE TO DO ALL TOGHETHER (PREPROCESSING FOLLOWED BY MODEL PREDICT)

# MULTIPLE PREDICTION
y_test_pred = pipeline.predict(X_test)

# EVALUATE MODEL FOR TEST ACCURACY SINCE WE HAVE TEST SET
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))


Accuracy: 0.8206229860365198

Classification Report:
               precision    recall  f1-score   support

       LEAVE       0.78      0.67      0.72       320
        STAY       0.84      0.90      0.87       611

    accuracy                           0.82       931
   macro avg       0.81      0.78      0.79       931
weighted avg       0.82      0.82      0.82       931


Confusion Matrix:
 [[214 106]
 [ 61 550]]


In [38]:
import dill

# save trained pipeline file

with open('pipeline.pkl', 'wb') as file:
    dill.dump(pipeline, file)

print('pipeline saved successfully to file')

pipeline saved successfully to file


In [39]:
import dill

#Load the saved pipeline from the file

with open('pipeline.pkl', 'rb') as file:
    loaded_pipeline = dill.load(file)

print('pipeline loaded successfully to file')

pipeline loaded successfully to file


In [40]:
loaded_pipeline.__getstate__()

{'steps': [('preprocessor',
   Pipeline(steps=[('preprocessor_stage_1',
                    Pipeline(steps=[('transform_PT',
                                     FunctionTransformer(func=<function <lambda> at 0x00000296A38DB920>))])),
                   ('preprocessor_stage_2',
                    ColumnTransformer(transformers=[('cat',
                                                     Pipeline(steps=[('OneHotEncode',
                                                                      OneHotEncoder(handle_unknown='ignore'))]),
                                                     ['EDUCATION', 'CITY',
                                                      'GENDER', 'EVERBENCHED',
                                                      'PAYMENTTIER']),
                                                    ('num',
                                                     Pipeline(steps=[('scale_data',
                                                                      StandardScaler()),
    

In [41]:
y_pred = loaded_pipeline.predict(my_pred_array)

y_pred

array(['STAY'], dtype=object)